Credit to https://radimrehurek.com/gensim/auto_examples/tutorials/run_fasttext.html#sphx-glr-download-auto-examples-tutorials-run-fasttext-py -- Gensim fastText model and use on Lee Corpus. FastText made by FB research

In [83]:
import operator
import numpy as np

In [1]:
%matplotlib inline

In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Here, we'll learn to work with fastText library for training word-embedding
models, saving & loading them and performing similarity operations & vector
lookups analogous to Word2Vec.



When to use FastText?
---------------------

The main principle behind `fastText <https://github.com/facebookresearch/fastText>`_ 
is that it treats each word as the aggregation of its subwords. Subwords are character n-grams of the word. The vector for a word is simply taken to be the sum of all vectors of its component char-ngrams.


According to a detailed comparison of Word2Vec and FastText in `this notebook <https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Word2Vec_FastText_Comparison.ipynb>`__, fastText does significantly better on syntactic tasks as compared to the original Word2Vec, especially when the size of the training corpus is small. Word2Vec slightly outperforms FastText on semantic tasks though. The differences grow smaller as the size of training corpus increases.


Training time for fastText is significantly higher than the Gensim version of Word2Vec (\ ``15min 42s`` vs ``6min 42s`` on text8, 17 mil tokens, 5 epochs, and a vector size of 100).


fastText can be used to obtain vectors for out-of-vocabulary (OOV) words, by summing up vectors for its component char-ngrams, provided at least one of the char-ngrams was present in the training data.




## Load Resume Data

In [3]:
import pandas as pd

# resumes only for now
# gives df with list of strings (tokenized) as well as lemmatized list of strings
local_resume_cleaned_lemmatized_tokenized_path = '/Users/richardkuzma/coding/NLP_projects/job_recommender_project/data/cleaned_lemmatized_tokenized_resume_dataset_maitrip.csv'

resumes = pd.read_csv(local_resume_cleaned_lemmatized_tokenized_path)
resumes.head()
resumes_sentences = resumes['lemmatized_resume'].tolist()

,ID,Category,dirty_resume,resume,tokenized_resume,lemmatized_resume
0,1,HR,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box...",john h smith phr po box callahan fl infog...,"['john', 'h', 'smith', 'phr', 'po', 'box', 'ca...","['john', 'h', 'smith', 'phr', 'po', 'box', 'ca..."
1,2,HR,b'Name Surname\nAddress\nMobile No/Email\nPERS...,name surname address mobile noemail personal p...,"['name', 'surname', 'address', 'mobile', 'noem...","['name', 'surname', 'address', 'mobile', 'noem..."
2,3,HR,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...,anthony brown hr assistant areas expertise per...,"['anthony', 'brown', 'hr', 'assistant', 'areas...","['anthony', 'brown', 'hr', 'assistant', 'area'..."
3,4,HR,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...,satheesh email id career objective pursue gro...,"['satheesh', 'email', 'id', 'career', 'objecti...","['satheesh', 'email', 'id', 'career', 'objecti..."
4,5,HR,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert...",human resources director expert organizational...,"['human', 'resources', 'director', 'expert', '...","['human', 'resource', 'director', 'expert', 'o..."


Training models
---------------




For the following examples, we'll use the Lee Corpus (which you already have if you've installed gensim) for training our model.






In [5]:

#this may be annoying later...
from pprint import pprint



from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath

# Set file names for train and test data
#corpus_file = datapath('lee_background.cor')



2020-04-16 13:44:26,423 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 13:44:26,425 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [6]:
model = FT_gensim(size=100)

# build the vocabulary
model.build_vocab(sentences = resumes_sentences)

# train the model
model.train(
    sentences=resumes_sentences, epochs=model.epochs,
    total_examples=model.corpus_count, total_words=model.corpus_total_words
)

print(model)

2020-04-16 13:44:37,433 : INFO : resetting layer weights
2020-04-16 13:45:04,598 : INFO : collecting all words and their counts
2020-04-16 13:45:04,674 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2020-04-16 13:45:04,711 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-04-16 13:45:06,740 : INFO : collected 31 word types from a corpus of 8591130 raw words and 1219 sentences
2020-04-16 13:45:06,746 : INFO : Loading a fresh vocabulary
2020-04-16 13:45:06,751 : INFO : effective_min_count=5 retains 31 unique words (100% of original 31, drops 0)
2020-04-16 13:45:06,767 : INFO : effective_min_count=5 leaves 8591130 word corpus (100% of original 8591130, drops 0)
2020-04-16 13:45:06,781 : INFO : deleting the raw counts dictionary of 31 items
2020-04-16 13:45:06,789 : INFO : sample=0.001 downsamples 26 most-common words
2020-04-16 13:45:06,796 : INFO : downsampling leaves est

FastText(vocab=31, size=100, alpha=0.025)


Training hyperparameters
^^^^^^^^^^^^^^^^^^^^^^^^




Hyperparameters for training the model follow the same pattern as Word2Vec. FastText supports the following parameters from the original word2vec:

- model: Training architecture. Allowed values: `cbow`, `skipgram` (Default `cbow`)
- size: Size of embeddings to be learnt (Default 100)
- alpha: Initial learning rate (Default 0.025)
- window: Context window size (Default 5)
- min_count: Ignore words with number of occurrences below this (Default 5)
- loss: Training objective. Allowed values: `ns`, `hs`, `softmax` (Default `ns`)
- sample: Threshold for downsampling higher-frequency words (Default 0.001)
- negative: Number of negative words to sample, for `ns` (Default 5)
- iter: Number of epochs (Default 5)
- sorted_vocab: Sort vocab by descending frequency (Default 1)
- threads: Number of threads to use (Default 12)


In addition, FastText has three additional parameters:

- min_n: min length of char ngrams (Default 3)
- max_n: max length of char ngrams (Default 6)
- bucket: number of buckets used for hashing ngrams (Default 2000000)


Parameters ``min_n`` and ``max_n`` control the lengths of character ngrams that each word is broken down into while training and looking up embeddings. If ``max_n`` is set to 0, or to be lesser than ``min_n``\ , no character ngrams are used, and the model effectively reduces to Word2Vec.



To bound the memory requirements of the model being trained, a hashing function is used that maps ngrams to integers in 1 to K. For hashing these character sequences, the `Fowler-Noll-Vo hashing function <http://www.isthe.com/chongo/tech/comp/fnv>`_ (FNV-1a variant) is employed.




**Note:** As in the case of Word2Vec, you can continue to train your model while using Gensim's native implementation of fastText.




## Now we have a fastText model trained on Resumes
<br>
Now we need to take individual job postings to compare

In [27]:
# load jobs into df

local_jobs_cleaned_lemmatized_tokenized_path = '/Users/richardkuzma/coding/NLP_projects/job_recommender_project/data/cleaned_lemmatized_tokenized_jobs_dataset_madhab.csv'

jobs = pd.read_csv(local_jobs_cleaned_lemmatized_tokenized_path)

In [28]:
jobs.head()

,Title,Company,JobDescription,RequiredQual,JobRequirement,label,combined,dirty_combined,tokenized_combined,lemmatized_combined
0,Chief Financial Officer,AMERIA Investment Consulting Company,AMERIA Investment Consulting Company is seekin...,"To perform this job successfully, an\r\nindivi...",- Supervises financial management and administ...,1,chief financial officer ameria investment cons...,Chief Financial Officer AMERIA Investment Cons...,"['chief', 'financial', 'officer', 'ameria', 'i...","['chief', 'financial', 'officer', 'ameria', 'i..."
1,Country Coordinator,Caucasus Environmental NGO Network (CENN),Public outreach and strengthening of a growing...,"- Degree in environmentally related field, or ...",- Working with the Country Director to provide...,2,country coordinator public outreach strengthen...,Country Coordinator Public outreach and streng...,"['country', 'coordinator', 'public', 'outreach...","['country', 'coordinator', 'public', 'outreach..."
2,BCC Specialist,Manoff Group,The LEAD (Local Enhancement and Development fo...,"- Advanced degree in public health, social sci...",- Identify gaps in knowledge and overseeing in...,3,bcc specialist lead local enhancement developm...,BCC Specialist The LEAD (Local Enhancement and...,"['bcc', 'specialist', 'lead', 'local', 'enhanc...","['bcc', 'specialist', 'lead', 'local', 'enhanc..."
3,"Community Development, Capacity Building and C...",Food Security Regional Cooperation and Stabili...,Food Security Regional Cooperation and Stabili...,- Higher Education and/or professional experie...,- Assist the Tavush Marz communities and commu...,4,community development capacity building confli...,"Community Development, Capacity Building and C...","['community', 'development', 'capacity', 'buil...","['community', 'development', 'capacity', 'buil..."
4,Country Economist (NOB),"United Nations Development Programme, Armenia",The United Nations Development Programme in Ar...,- Minimum Masters Degree in Economics;\r\n- Mi...,The incumbent under direct supervision of UNDP...,5,country economist nob united nations developme...,Country Economist (NOB) The United Nations Dev...,"['country', 'economist', 'nob', 'united', 'nat...","['country', 'economist', 'nob', 'united', 'nat..."


In [29]:
jobs_list = jobs['lemmatized_combined'].tolist()

In [30]:
jobs.shape[0]

13124

In [84]:
def pick_job():
    print("There are {} jobs".format(jobs.shape[0]))
    
    # Select a random int from 0 to length of rjob set
    rand_int = np.random.randint(1, jobs.shape[0]+1)
    
    
    
    # selection = 2
    selection = rand_int
    
    
    
    print ('\nselected job is ID #{}'.format(selection))
    
    # pick the job text and ID associated with the random int
    job_label = jobs.iloc[selection - 1, jobs.columns.get_loc('label')] #we could grab ID, but this works for non-indexed labels too
    job_title = jobs.iloc[selection - 1 ]['Title']
    job_company = jobs.iloc[selection - 1 ]['Company']
    job_description = jobs.iloc[selection - 1 ]['JobDescription']
    
    
    print('Job Posting ID is: {}'.format(job_label))
    print('Job Posting Title: {}'.format(job_title))
    print('Job Posting Company: {}'.format(job_company))
    print('Job Posting Description: {}'.format(job_description))
    
    #Convert the sample document into a list and use the infer_vector method to get a vector representation for it
    job_text_to_process = jobs['lemmatized_combined'][selection - 1]
    
    return job_text_to_process


In [96]:
def given_job_find_similar_resumes(job_you_pick):
  
    #find all distances between chosen job and each resume
    temp_distance = []
    min_dist = float("inf")
    min_index = float("inf")


    # for i in range len(resume_sentences): 
    for i in range (0, len(resumes_sentences)):
        dist = model.wmdistance(job_you_pick, resumes_sentences[i])
        if dist < min_dist:
            min_dist = dist
            min_index = i
        temp_distance.append((dist, i))

    #sort list of tuples
    temp_distance.sort(key = operator.itemgetter(0))
    temp_distance

    num_similar = 10 #or 10, 20, 25
    print('\nPrinting {} most similar candidates...\n'.format(num_similar))
    for i in range(0,num_similar):
        print('\n#{} most similar job'.format(i+1))
        print('Resume ID from list: {}'.format(temp_distance[i][1]))
        print('WM Distance: {}'.format(temp_distance[i][0]))
        print('Resume ID from df: {}'.format(resumes.iloc[temp_distance[i][1]]['ID']))
        print('Resume text (500 chars): {}'.format(resumes.iloc[temp_distance[i][1]]['resume'][0:500]))


In [100]:
chosen_job = pick_job()

There are 13124 jobs

selected job is ID #3187
Job Posting ID is: 3187
Job Posting Title: PR Manager
Job Posting Company: Megafood LLC
Job Posting Description: Megafood LLC is seeking a candidate for the position
of PR Manager who will coordinate the company's market research,
marketing stratagy, product development and public relations activities.
The PR Manager will initiate, develop and manage PR programs to support
and enhance the overall marketing, sales and business goals of the
company.


In [101]:
given_job_find_similar_resumes(chosen_job)

/Users/richardkuzma/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  # This is added back by InteractiveShellApp.init_path()
2020-04-16 14:39:59,608 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:39:59,614 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 7820 corpus positions)
2020-04-16 14:39:59,634 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:39:59,637 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 2855 corpus positions)
2020-04-16 14:39:59,659 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:39:59,663 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 8309 corpus positions)
2020-04-16 14:39:59,694 : INFO : addi

2020-04-16 14:40:01,059 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 60456 corpus positions)
2020-04-16 14:40:01,098 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:01,100 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 5285 corpus positions)
2020-04-16 14:40:01,134 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:01,138 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 11989 corpus positions)
2020-04-16 14:40:01,165 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:01,168 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3374 corpus positions)
2020-04-16 14:40:01,193 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:01,196 : INFO : built Dictionary(31 unique tokens: ['

2020-04-16 14:40:02,603 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:02,610 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4899 corpus positions)
2020-04-16 14:40:02,644 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:02,650 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4389 corpus positions)
2020-04-16 14:40:02,727 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:02,729 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 2981 corpus positions)
2020-04-16 14:40:02,767 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:02,776 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 14692 corpus positions)
2020-04-16 14:40:02,838 : INFO : adding document #0 to Dictionary(0 uni

2020-04-16 14:40:04,379 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 14060 corpus positions)
2020-04-16 14:40:04,430 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:04,433 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4300 corpus positions)
2020-04-16 14:40:04,461 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:04,486 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4800 corpus positions)
2020-04-16 14:40:04,530 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:04,534 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 10546 corpus positions)
2020-04-16 14:40:04,571 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:04,573 : INFO : built Dictionary(31 unique tokens: ['

2020-04-16 14:40:05,716 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:05,719 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 8922 corpus positions)
2020-04-16 14:40:05,752 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:05,756 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 9691 corpus positions)
2020-04-16 14:40:05,787 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:05,790 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3721 corpus positions)
2020-04-16 14:40:05,826 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:05,838 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 27199 corpus positions)
2020-04-16 14:40:05,881 : INFO : adding document #0 to Dictionary(0 uni

...

2020-04-16 14:40:55,193 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 5572 corpus positions)
2020-04-16 14:40:55,236 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:55,243 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 5626 corpus positions)
2020-04-16 14:40:55,278 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:55,281 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 8698 corpus positions)
2020-04-16 14:40:55,313 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:55,320 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3091 corpus positions)
2020-04-16 14:40:55,344 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:55,349 : INFO : built Dictionary(31 unique tokens: [' '

2020-04-16 14:40:56,789 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:56,791 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 2985 corpus positions)
2020-04-16 14:40:56,822 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:56,827 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 2775 corpus positions)
2020-04-16 14:40:56,857 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:56,859 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3495 corpus positions)
2020-04-16 14:40:56,900 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:56,902 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3138 corpus positions)
2020-04-16 14:40:56,935 : INFO : adding document #0 to Dictionary(0 uniq

2020-04-16 14:40:58,204 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 6579 corpus positions)
2020-04-16 14:40:58,244 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:58,269 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 1762 corpus positions)
2020-04-16 14:40:58,324 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:58,326 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3209 corpus positions)
2020-04-16 14:40:58,370 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:58,373 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3422 corpus positions)
2020-04-16 14:40:58,419 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:58,421 : INFO : built Dictionary(31 unique tokens: [' '

2020-04-16 14:40:59,770 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:59,772 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4181 corpus positions)
2020-04-16 14:40:59,802 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:59,804 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3147 corpus positions)
2020-04-16 14:40:59,844 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:59,857 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 36454 corpus positions)
2020-04-16 14:40:59,890 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:40:59,894 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4199 corpus positions)
2020-04-16 14:40:59,932 : INFO : adding document #0 to Dictionary(0 uni


Printing 10 most similar candidates...


#1 most similar job
Resume ID from list: 692
WM Distance: 0.16037376807236195
Resume ID from df: 693
Resume text (500 chars): matt arnzen mattmattarnzencom    portland professional summary marketing professional fifteen years progressive experience digital marketing product management user experience design accomplished creative development website operations digital platform management collaborating effectively stakeholders vendors clients digital marketing product management content management marketing automation website design development search engine optimization web analytics reporting customer relationship mana

#2 most similar job
Resume ID from list: 437
WM Distance: 0.16067475820688432
Resume ID from df: 438
Resume text (500 chars): steve jones business operations manager dayjob ltd big peg birmingham b nf    e infodayjobcom personal statment e infodayjobcom progressive business operations manager particular strength driving performa